### Importing Module

In [47]:
from __future__ import division

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

import os
import random
import cv2
import math
import numpy as np
import scipy
import scipy.misc
from scipy import pi
from subprocess import call
from datetime import datetime
from itertools import islice
import matplotlib.pyplot as plt 
import tensorflow as tf

### Loading Data

In [48]:
image_data = []
angle_data = []

# Get number of images
num_images = 0

# Number of images for training
num_train_images = 0

# Number of images for testing
num_test_images = 0

def load_dataset():
    # Read data.txt
    with open("/Users/mohdsaquib/downloads/autopilot/driving_dataset/data.txt") as fp:
        for line in fp:
            image_data.append("/Users/mohdsaquib/downloads/autopilot/driving_dataset/" + line.split()[0])

            # the paper by Nvidia uses the inverse of the turning radius,
            # but steering wheel angle is proportional to the inverse of turning radius
            # so the steering wheel angle in radians is used as the output       
            angle_data.append(float(line.split()[1]) * scipy.pi / 180)

def split_dataset(train_split,test_split):
    images_to_train = image_data[:int(len(image_data) * train_split)]    
    angles_to_train = angle_data[:int(len(image_data) * train_split)]

    images_to_test = image_data[-int(len(image_data) * test_split):]
    angles_to_test = angle_data[-int(len(image_data) * test_split):]
    
    return images_to_train,angles_to_train,images_to_test,angles_to_test


In [49]:
# Load dataset
load_dataset()

# Split dataset
images_to_train,angles_to_train,images_to_test,angles_to_test = split_dataset(0.8,0.2)

num_images = len(image_data)
print("Total number of images: ",num_images)

num_train_images = len(images_to_train)
print("Total number of images for training: ",num_train_images)

num_test_images = len(images_to_test)
print("Total number of images for testing: ",num_test_images)

Total number of images:  45406
Total number of images for training:  36324
Total number of images for testing:  9081


### Base Line Model

In [52]:
#Model 1: Base line Model: y_test_pred = mean(y_train_i) 
train_mean_angle = np.mean(angles_to_train)

print('Test_MSE(MEAN):%f' % np.mean(np.square(angles_to_test - train_mean_angle)))

Test_MSE(MEAN):0.191142


### Model

In [55]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W, stride):
    return tf.nn.conv2d(x, W, strides=[1, stride, stride, 1], padding='VALID')

In [56]:
true_image = tf.placeholder(tf.float32, shape=[None, 66, 200, 3],name="true_image")
true_angle = tf.placeholder(tf.float32, shape=[None, 1],name="true_angle")

x_image = true_image

#first convolutional layer
W_conv1 = weight_variable([5, 5, 3, 24])
b_conv1 = bias_variable([24])

h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1, 2) + b_conv1)

#second convolutional layer
W_conv2 = weight_variable([5, 5, 24, 36])
b_conv2 = bias_variable([36])

h_conv2 = tf.nn.relu(conv2d(h_conv1, W_conv2, 2) + b_conv2)

#third convolutional layer
W_conv3 = weight_variable([5, 5, 36, 48])
b_conv3 = bias_variable([48])

h_conv3 = tf.nn.relu(conv2d(h_conv2, W_conv3, 2) + b_conv3)

#fourth convolutional layer
W_conv4 = weight_variable([3, 3, 48, 64])
b_conv4 = bias_variable([64])

h_conv4 = tf.nn.relu(conv2d(h_conv3, W_conv4, 1) + b_conv4)

#fifth convolutional layer
W_conv5 = weight_variable([3, 3, 64, 64])
b_conv5 = bias_variable([64])

h_conv5 = tf.nn.relu(conv2d(h_conv4, W_conv5, 1) + b_conv5)

#FCL 1
W_fc1 = weight_variable([1152, 1164])
b_fc1 = bias_variable([1164])

h_conv5_flat = tf.reshape(h_conv5, [-1, 1152])
h_fc1 = tf.nn.relu(tf.matmul(h_conv5_flat, W_fc1) + b_fc1)

keep_prob = tf.placeholder(tf.float32,name="keep_prob")
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

#FCL 2
W_fc2 = weight_variable([1164, 100])
b_fc2 = bias_variable([100])

h_fc2 = tf.nn.relu(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

h_fc2_drop = tf.nn.dropout(h_fc2, keep_prob)

#FCL 3
W_fc3 = weight_variable([100, 50])
b_fc3 = bias_variable([50])#FCL 3
W_fc3 = weight_variable([100, 50])
b_fc3 = bias_variable([50])

h_fc3 = tf.nn.relu(tf.matmul(h_fc2_drop, W_fc3) + b_fc3)

h_fc3_drop = tf.nn.dropout(h_fc3, keep_prob)

#FCL 4
W_fc4 = weight_variable([50, 10])
b_fc4 = bias_variable([10])

h_fc4 = tf.nn.relu(tf.matmul(h_fc3_drop, W_fc4) + b_fc4)

h_fc4_drop = tf.nn.dropout(h_fc4, keep_prob)

#Output
W_fc5 = weight_variable([10, 1])
b_fc5 = bias_variable([1])

# atan activation function with scaling
predicted_angle = tf.multiply(tf.atan(tf.matmul(h_fc4_drop, W_fc5) + b_fc5), 2)
predicted_angle = tf.identity(predicted_angle,name="predicted_angle")

### Creating batch for training dataset

In [72]:
#points to the end of the last batch
train_batch_pointer = 0
test_batch_pointer = 0

# Utility Functions
def LoadTrainBatch(batch_size):
    global train_batch_pointer
    x_out = []
    y_out = []
    for i in range(0, batch_size):
        x_out.append(scipy.misc.imresize(scipy.misc.imread(images_to_train[(train_batch_pointer + i) % num_train_images])[-150:], 
                                         [66, 200]) / 255.0)
        y_out.append([angles_to_train[(train_batch_pointer + i) % num_train_images]])
    train_batch_pointer += batch_size
    return x_out, y_out

def LoadTestBatch(batch_size): 
    global test_batch_pointer
    x_out = []
    y_out = []
    for i in range(0, batch_size):
        x_out.append(scipy.misc.imresize(scipy.misc.imread(images_to_test[(test_batch_pointer + i) % num_test_images])[-150:], 
                                         [66, 200]) / 255.0)
        y_out.append([angles_to_test[(test_batch_pointer + i) % num_test_images]])
    test_batch_pointer += batch_size
    return x_out, y_out

### Saving the model in log directory

In [73]:
LOGDIR = './models/atan/'

# Lets start the tensorflow session
sess = tf.InteractiveSession()

/anaconda3/lib/python3.6/site-packages/tensorflow/python/client/session.py:1714: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


### Training

In [61]:
start = datetime.now()

print("Let the model learn itself...")
print()

L2NormConst = 0.001

train_vars = tf.trainable_variables()

loss = tf.reduce_mean(tf.square(tf.subtract(true_angle, predicted_angle))) + tf.add_n([tf.nn.l2_loss(v) for v in train_vars]) * L2NormConst
train_step = tf.train.AdamOptimizer(1e-4).minimize(loss)

sess.run(tf.global_variables_initializer())

# create a summary to monitor cost tensor
tf.summary.scalar("loss", loss)

# merge all summaries into a single op
merged_summary_op =  tf.summary.merge_all()

saver = tf.train.Saver()

# op to write logs to Tensorboard
logs_path = './logs'
summary_writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())

epochs = 30
batch_size = 100

# train over the dataset about 30 times
previous_i = 0
previous_loss = 0
for epoch in range(epochs):
    for i in range(int(num_images/batch_size)):        
        xs, ys = LoadTrainBatch(batch_size)
        train_step.run(feed_dict={true_image: xs, true_angle: ys, keep_prob: 0.80})
        if i % 10 == 0:            
            xs, ys = LoadTestBatch(batch_size)
            loss_value = loss.eval(feed_dict={true_image:xs, true_angle: ys, keep_prob: 1.0})
            previous_loss = loss_value
            previous_i = i
            # print("Epoch: %d, Step: %d, Loss: %g" % (epoch, epoch * batch_size + i, loss_value))

        # write logs at every iteration
        summary = merged_summary_op.eval(feed_dict={true_image:xs, true_angle: ys, keep_prob: 1.0})
        summary_writer.add_summary(summary, epoch * num_images/batch_size + i)

        if i % batch_size == 0:
            if not os.path.exists(LOGDIR):
                os.makedirs(LOGDIR)            
            checkpoint_path = os.path.join(LOGDIR, "model_atan.ckpt")
            filename = saver.save(sess, checkpoint_path)    
    print("Epoch: %d, Step: %d, Loss: %g" % (epoch, epoch * batch_size + previous_i, previous_loss)) 
    print("Model saved in file: %s" % filename)
    print()

print("Run the command line:\n" \
          "--> tensorboard --logdir=./logs " \
          "\nThen open http://0.0.0.0:6006/ into your web browser")

print("\nTime taken to train the model: ",datetime.now() - start)

Let the model learn itself...

Epoch: 0, Step: 450, Loss: 12.2221
Model saved in file: ./models/atan/model_atan.ckpt

Epoch: 1, Step: 550, Loss: 6.7393
Model saved in file: ./models/atan/model_atan.ckpt

Epoch: 2, Step: 650, Loss: 3.5296
Model saved in file: ./models/atan/model_atan.ckpt

Epoch: 3, Step: 750, Loss: 2.57659
Model saved in file: ./models/atan/model_atan.ckpt

Epoch: 4, Step: 850, Loss: 1.60101
Model saved in file: ./models/atan/model_atan.ckpt

Epoch: 5, Step: 950, Loss: 1.38056
Model saved in file: ./models/atan/model_atan.ckpt

Epoch: 6, Step: 1050, Loss: 1.03843
Model saved in file: ./models/atan/model_atan.ckpt

Epoch: 7, Step: 1150, Loss: 0.89442
Model saved in file: ./models/atan/model_atan.ckpt

Epoch: 8, Step: 1250, Loss: 0.82543
Model saved in file: ./models/atan/model_atan.ckpt

Epoch: 9, Step: 1350, Loss: 0.680471
Model saved in file: ./models/atan/model_atan.ckpt

Epoch: 10, Step: 1450, Loss: 0.605669
Model saved in file: ./models/atan/model_atan.ckpt

Epoch:

In [74]:
# Lets close the tensorflow session
sess.close()

### Enter python3 run_dataset.py in your command prompt or terminal